In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.metrics import classification_report

# Load your dataset
data = pd.read_csv("C:/Users/VIDYA RAJU/Downloads/depdata.csv")  



# Label encode the target variable
le = LabelEncoder()
data['Department_encoded'] = le.fit_transform(data['Department'])
num_departments = len(data['Department'].unique())

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Symptoms'])
X = tokenizer.texts_to_sequences(data['Symptoms'])
max_len = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_len, padding='post')

# Convert labels to one-hot encoded format
y = pd.get_dummies(data['Department_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_len)(input_layer)
conv1d_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv1d_layer)
output_layer = Dense(num_departments, activation='softmax')(pooling_layer)

cnn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = cnn_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)

# Print classification report
print("\nClassification Report:")
print(classification_report(np.argmax(y_test, axis=-1), y_pred))


Epoch 1/50
4/4 [==============================] - 2s 277ms/step - loss: 2.1939 - accuracy: 0.1328 - val_loss: 2.1705 - val_accuracy: 0.2727
Epoch 2/50
4/4 [==============================] - 0s 80ms/step - loss: 2.1476 - accuracy: 0.2500 - val_loss: 2.1557 - val_accuracy: 0.2727
Epoch 3/50
4/4 [==============================] - 0s 35ms/step - loss: 2.1097 - accuracy: 0.3984 - val_loss: 2.1408 - val_accuracy: 0.3939
Epoch 4/50
4/4 [==============================] - 0s 44ms/step - loss: 2.0736 - accuracy: 0.6094 - val_loss: 2.1241 - val_accuracy: 0.3939
Epoch 5/50
4/4 [==============================] - 1s 187ms/step - loss: 2.0367 - accuracy: 0.7500 - val_loss: 2.1037 - val_accuracy: 0.4242
Epoch 6/50
4/4 [==============================] - 0s 69ms/step - loss: 1.9972 - accuracy: 0.7656 - val_loss: 2.0818 - val_accuracy: 0.4242
Epoch 7/50
4/4 [==============================] - 0s 55ms/step - loss: 1.9533 - accuracy: 0.7656 - val_loss: 2.0545 - val_accuracy: 0.4242
Epoch 8/50
4/4 [=========

In [2]:
# Read the symptom from a text file
with open("C:/Users/VIDYA RAJU/Downloads/symptom.txt", "r") as file:
    new_symptoms = file.read()

# Tokenize and pad the new symptom paragraph
new_X = tokenizer.texts_to_sequences([new_symptoms])
new_X = pad_sequences(new_X, maxlen=max_len, padding='post')

# Predict the department
predicted_department = cnn_model.predict(new_X)
predicted_department = le.inverse_transform([np.argmax(predicted_department)])

print("Predicted Department:", predicted_department)

1/1 [==============================] - 0s 18ms/step
Predicted Department: ['Hematology']
